In [1]:
from kubernetes import client, config

# Configs can be set in Configuration class directly or using helper utility
config.load_kube_config()

v1 = client.CoreV1Api()
print("Listing pods with their IPs:")
ret = v1.list_namespaced_pod("social-network")
for i in ret.items:
    print("%s\t%s\t%s" % (i.status.pod_ip, i.metadata.namespace, i.metadata.name))

Listing pods with their IPs:
10.244.0.103	social-network	compose-post-service-866f6d7b74-hl962
10.244.0.96	social-network	home-timeline-redis-7994f7cb85-d89c9
10.244.0.88	social-network	home-timeline-service-bbc9bdd7b-cndcg
10.244.0.95	social-network	jaeger-7455bdbc47-jchjt
10.244.0.113	social-network	media-frontend-7f4999dcd-n64xn
10.244.0.109	social-network	media-memcached-7c887f8cc6-l5q56
10.244.0.101	social-network	media-mongodb-cb6857b9f-gwf5w
10.244.0.107	social-network	media-service-7848df64c4-74h8s
10.244.0.112	social-network	nginx-thrift-57fdb4c4f9-qn6tv
10.244.0.99	social-network	post-storage-memcached-675b5c57bd-smb49
10.244.0.102	social-network	post-storage-mongodb-5d7bfcd455-t6sdz
10.244.0.104	social-network	post-storage-service-84b659876c-7tt26
10.244.0.87	social-network	social-graph-mongodb-78b95799d7-856w7
10.244.0.93	social-network	social-graph-redis-7d9b494f66-px74j
10.244.0.91	social-network	social-graph-service-55b7888dc7-b6jzj
10.244.0.111	social-network	text-servi

In [2]:
api_client = client.ApiClient()
current_node = 0
node_names = list(map(lambda x: x.metadata.name, v1.list_node().items))
node_names

['minikube']

In [3]:
api_client = client.ApiClient()
current_node = 0
node_names = ["minikube"]
response = api_client.call_api(f"/api/v1/nodes/{node_names[current_node]}/proxy/metrics/resource", "GET", response_type="str")
data = str(response[0])
rows = data.split("\n")
rows

['# HELP container_cpu_usage_seconds_total [ALPHA] Cumulative cpu time consumed by the container in core-seconds',
 '# TYPE container_cpu_usage_seconds_total counter',
 'container_cpu_usage_seconds_total{container="compose-post-service",namespace="social-network",pod="compose-post-service-866f6d7b74-hl962"} 0.041902 1712040223306',
 'container_cpu_usage_seconds_total{container="coredns",namespace="kube-system",pod="coredns-5d78c9869d-lcldh"} 10.812828 1712040224189',
 'container_cpu_usage_seconds_total{container="etcd",namespace="kube-system",pod="etcd-minikube"} 108.664146 1712040224398',
 'container_cpu_usage_seconds_total{container="home-timeline-redis",namespace="social-network",pod="home-timeline-redis-7994f7cb85-d89c9"} 11.123847 1712040225225',
 'container_cpu_usage_seconds_total{container="home-timeline-service",namespace="social-network",pod="home-timeline-service-bbc9bdd7b-cndcg"} 0.051963 1712040225143',
 'container_cpu_usage_seconds_total{container="jaeger",namespace="socia

In [16]:
# class Pods:
#     def __init__(self):
#         self.data = {}

#     def __str__(self):
#         return str(data)
    
#     def __repr__(self):
#         return str(data)
    
#     def add_cpu(self, name, namespace, timestamp, value):
#         try:
#             self.data[name]["cpu"] = value
#             self.data[name]["cpu_timestamp"] = timestamp
#         except KeyError:
#             self.data[name] = {"namespace": namespace, "pod": name, "cpu": value, "cpu_timestamp": timestamp}

#     def add_memory(self, name, namespace, timestamp, value):
#         try:
#             self.data[name]["memory"] = value
#             self.data[name]["memory_timestamp"] = timestamp
#         except KeyError:
#             self.data[name] = {"namespace": namespace, "pod": name, "memory": value, "memory_timestamp": timestamp}

# class Nodes:
#     def __init__(self):
#         self.data = {}

#     def __str__(self):
#         return str(data)
    
#     def __repr__(self):
#         return str(data)
    
#     def add_cpu(self, name, timestamp, value):
#         try:
#             self.data[name]["cpu"] = value
#             self.data[name]["cpu_timestamp"] = timestamp
#         except KeyError:
#             self.data[name] = {"cpu": value, "cpu_timestamp": timestamp}

#     def add_memory(self, name, timestamp, value):
#         try:
#             self.data[name]["memory"] = value
#             self.data[name]["memory_timestamp"] = timestamp
#         except KeyError:
#             self.data[name] = {"memory": value, "memory_timestamp": timestamp}

# class Containers:
#     def __init__(self):
#         self.data = {}

#     def __str__(self):
#         return str(data)
    
#     def __repr__(self):
#         return str(data)
    
#     def add_cpu(self, name, namespace, pod, timestamp, value):
#         try:
#             self.data[pod]["cpu"] = value
#             self.data[pod]["cpu_timestamp"] = timestamp
#         except KeyError:
#             self.data[pod] = {"namespace": namespace, "pod": pod, "container": name, "cpu": value, "cpu_timestamp": timestamp}

#     def add_memory(self, name, namespace, pod, timestamp, value):
#         try:
#             self.data[pod]["memory"] = value
#             self.data[pod]["memory_timestamp"] = timestamp
#         except KeyError:
#             self.data[pod] = {"namespace": namespace, "pod": pod, "container": name, "memory": value, "memory_timestamp": timestamp}

In [5]:
data = str(response[0])
rows = data.split("\n")

# Remove comments and empty lines
rows = [row for row in rows if not row.startswith("#") and row.strip() != ""]

# Split lines from rows
rows = [row.split() for row in rows]
from model import PodMetric, ContainerMetric, NodeMetric
from typing import Dict

def process_pod_or_container(row: list[str], pods: Dict[str, PodMetric], containers: Dict[str, ContainerMetric]):
    raw_name = row[0]
    value = row[1]
    timestamp = row[2]
    raw_type = raw_name.split("{")[0]
    pod = raw_name.split("\"")[-2].split("\"")[-1]
    namespace = raw_name.split("\"")[-4].split("\"")[-1]
    container = raw_name.split("\"")[1].split("\"")[0]
    match raw_type:
        case "container_cpu_usage_seconds_total":
            try:
                containers[pod].cpu_usage = value
                containers[pod].cpu_timestamp = timestamp
            except KeyError:
                containers[pod] = ContainerMetric(namespace=namespace, pod=pod, container=container)
                containers[pod].cpu_usage = value
                containers[pod].cpu_timestamp = timestamp
        case "container_memory_working_set_bytes":
            try:
                containers[pod].memory_usage = value
                containers[pod].memory_timestamp = timestamp
            except KeyError:
                containers[pod] = ContainerMetric(namespace=namespace, pod=pod, container=container)
                containers[pod].memory_usage = value
                containers[pod].memory_timestamp = timestamp
        case "pod_memory_working_set_bytes":
            try:
                pods[pod].memory_usage = value
                pods[pod].memory_timestamp = timestamp
            except KeyError:
                pods[pod] = PodMetric(namespace=namespace, pod=pod)
                pods[pod].memory_usage = value
                pods[pod].memory_timestamp = timestamp
        case "pod_cpu_usage_seconds_total":
            try:
                pods[pod].cpu_usage = value
                pods[pod].cpu_timestamp = timestamp
            except KeyError:
                pods[pod] = PodMetric(namespace=namespace, pod=pod)
                pods[pod].cpu_usage = value
                pods[pod].cpu_timestamp = timestamp


def process_node(row: list[str], nodes: Dict[str, NodeMetric], node_name: str):
    raw_type = row[0]
    value = row[1]
    timestamp = row[2]
    match raw_type:
        case "node_memory_working_set_bytes":
            try:
                nodes[node_names[current_node]].memory_usage = value 
                nodes[node_names[current_node]].memory_timestamp = timestamp 

            except KeyError:
                nodes[node_names[current_node]] = NodeMetric(name=node_name)
                nodes[node_names[current_node]].memory_usage = value
                nodes[node_names[current_node]].memory_timestamp = timestamp
        case "node_cpu_usage_seconds_total":
            try:
                nodes[node_names[current_node]].cpu_usage = value 
                nodes[node_names[current_node]].cpu_timestamp = timestamp 
            except KeyError:
                nodes[node_names[current_node]] = NodeMetric(name=node_name)
                nodes[node_names[current_node]].cpu_usage = value
                nodes[node_names[current_node]].cpu_timestamp = timestamp


from db import insert_batch_container_data, insert_batch_node_data, insert_batch_pod_data, connect

def test():
    pods = {}
    nodes = {}
    containers = {}
    node_name = "minikube"
    for row in rows:
        try:
            if row[0] == "scrape_error":
                error_count = row[1]
            elif row[0][:4] == "node":
                process_node(row, nodes, node_name)
            else:
                process_pod_or_container(row, pods, containers)
                
        except IndexError:
            print("Error:",row)
    
    conn = connect()
    insert_batch_pod_data(conn, pods)
    insert_batch_container_data(conn, containers)
    insert_batch_node_data(conn, nodes)
    conn.close()

test()

PodMetric(pod='coredns-5d78c9869d-lcldh', namespace='kube-system', cpu_usage='10.829746', memory_usage='3.2428032e+07', cpu_timestamp='1712040220801', memory_timestamp='1712040220801')
PodMetric(pod='etcd-minikube', namespace='kube-system', cpu_usage='108.557418', memory_usage='8.357888e+07', cpu_timestamp='1712040214193', memory_timestamp='1712040214193')
PodMetric(pod='kube-apiserver-minikube', namespace='kube-system', cpu_usage='219.824696', memory_usage='3.49175808e+08', cpu_timestamp='1712040219870', memory_timestamp='1712040219870')
PodMetric(pod='kube-controller-manager-minikube', namespace='kube-system', cpu_usage='78.786472', memory_usage='1.04873984e+08', cpu_timestamp='1712040217851', memory_timestamp='1712040217851')
PodMetric(pod='kube-proxy-p8wcq', namespace='kube-system', cpu_usage='2.409796', memory_usage='3.0519296e+07', cpu_timestamp='1712040220774', memory_timestamp='1712040220774')
PodMetric(pod='kube-scheduler-minikube', namespace='kube-system', cpu_usage='15.49808

In [27]:
rows

[['container_cpu_usage_seconds_total{container="cast-info-memcached",namespace="media-microservices",pod="cast-info-memcached-56b648cb4-274nc"}',
  '1.863737',
  '1709119445656'],
 ['container_cpu_usage_seconds_total{container="cast-info-mongodb",namespace="media-microservices",pod="cast-info-mongodb-566f5887c-vqzp6"}',
  '67.178791',
  '1709119445662'],
 ['container_cpu_usage_seconds_total{container="cast-info-service",namespace="media-microservices",pod="cast-info-service-7ccdbff8-zf45m"}',
  '2.303045',
  '1709119446795'],
 ['container_cpu_usage_seconds_total{container="compose-review-memcached",namespace="media-microservices",pod="compose-review-memcached-598d9dc6cd-cq2dm"}',
  '69.726172',
  '1709119445446'],
 ['container_cpu_usage_seconds_total{container="compose-review-service",namespace="media-microservices",pod="compose-review-service-76ccb99f59-dgdcn"}',
  '311.354159',
  '1709119446675'],
 ['container_cpu_usage_seconds_total{container="coredns",namespace="kube-system",pod="c

In [42]:
containers

{'cast-info-memcached-56b648cb4-274nc': {'namespace': 'media-microservices',
  'container': 'cast-info-memcached',
  'pod': 'cast-info-memcached-56b648cb4-274nc',
  'cpu': '1.831069',
  'cpu_timestamp': '1709119067945',
  'memory_timestamp': '1709119059484'},
 'cast-info-mongodb-566f5887c-vqzp6': {'namespace': 'media-microservices',
  'container': 'cast-info-mongodb',
  'pod': 'cast-info-mongodb-566f5887c-vqzp6',
  'cpu': '65.995307',
  'cpu_timestamp': '1709119068060',
  'memory_timestamp': '1709119066777'},
 'cast-info-service-7ccdbff8-zf45m': {'namespace': 'media-microservices',
  'container': 'cast-info-service',
  'pod': 'cast-info-service-7ccdbff8-zf45m',
  'cpu': '2.297739',
  'cpu_timestamp': '1709119069107',
  'memory_timestamp': '1709119053932'},
 'compose-review-memcached-598d9dc6cd-cq2dm': {'namespace': 'media-microservices',
  'container': 'compose-review-memcached',
  'pod': 'compose-review-memcached-598d9dc6cd-cq2dm',
  'cpu': '69.678921',
  'cpu_timestamp': '17091190680

In [5]:
pods

{'coredns-5dd5756b68-xpdq5': {'cpu': '11.362751',
  'namespace': 'kube-system',
  'timestamp': '1709109823228',
  'pod': 'coredns-5dd5756b68-xpdq5',
  'memory': '3.0539776e+07'},
 'etcd-minikube': {'cpu': '112.087974',
  'namespace': 'kube-system',
  'timestamp': '1709109815866',
  'pod': 'etcd-minikube',
  'memory': '8.2300928e+07'},
 'kube-apiserver-minikube': {'cpu': '288.816945',
  'namespace': 'kube-system',
  'timestamp': '1709109824472',
  'pod': 'kube-apiserver-minikube',
  'memory': '2.83435008e+08'},
 'kube-controller-manager-minikube': {'cpu': '85.602686',
  'namespace': 'kube-system',
  'timestamp': '1709109816513',
  'pod': 'kube-controller-manager-minikube',
  'memory': '6.4032768e+07'},
 'kube-proxy-ht6xb': {'cpu': '2.988617',
  'namespace': 'kube-system',
  'timestamp': '1709109825232',
  'pod': 'kube-proxy-ht6xb',
  'memory': '3.4811904e+07'},
 'kube-scheduler-minikube': {'cpu': '17.420849',
  'namespace': 'kube-system',
  'timestamp': '1709109815429',
  'pod': 'kube-s

In [25]:
import pandas as pd


df = pd.DataFrame(pods.values())

41
41


In [18]:
# Delete the csv files
import os
os.remove("pods.csv")
os.remove("error_count.csv")
os.remove("containers.csv")
os.remove("nodes.csv")

In [25]:
import sched, time, os
import pandas as pd
from multiprocessing import Pool

# Create empty csv files

def query_a_node(node_name: str): 
    print("Doing stuff...")

    # Create a dictionary with the data
    pods = {}
    containers = {}
    nodes = {}
    error_count = 0
    response = api_client.call_api(f"/api/v1/nodes/{node_name}/proxy/metrics/resource", "GET", response_type="str")
    data = str(response[0])
    rows = data.split("\n")

    # Remove comments and empty lines
    rows = [row for row in rows if not row.startswith("#") and row.strip() != ""]

    # Split lines from rows
    rows = [row.split() for row in rows]

    for row in rows:
        try:
            if row[0] == "scrape_error":
                error_count = row[1]
            elif row[0][:4] == "node":
                process_node(row, nodes)
            else:
                process_pod_or_container(row, pods, containers)
                
        except IndexError:
            print("Error:",row)

    pods_df = pd.DataFrame(pods.values())
    pods_df.to_csv("pods.csv", mode='a', header=not os.path.exists("pods.csv"), index=False)

    containers_df = pd.DataFrame(containers.values())
    containers_df.to_csv("containers.csv", mode='a', header=not os.path.exists("containers.csv"), index=False)

    nodes_df = pd.DataFrame(nodes.values())
    nodes_df.to_csv("nodes.csv", mode='a', header=not os.path.exists("nodes.csv"), index=False)

    error_count_df = pd.DataFrame({"error_count": [error_count], "timestamp": [time.time()]})
    error_count_df.to_csv("error_count.csv", mode='a', header=not os.path.exists("error_count.csv"), index=False)

    print("Done")

def query_all_nodes(scheduler):
    # schedule the next call first
    scheduler.enter(5, 1, query_all_nodes, (scheduler,))
    pool = Pool(processes=len(node_names))
    for node in node_names:
        pool.apply_async(query_a_node, (node,))
    pool.close()
    pool.join()

my_scheduler = sched.scheduler(time.time, time.sleep)
my_scheduler.enter(0, 1, query_all_nodes, (my_scheduler,))
my_scheduler.run()

Doing stuff...


Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...
Done
Doing stuff...


KeyboardInterrupt: 

In [14]:
node_name = "minikube"
import pandas as pd
import os
import time
# Create a dictionary with the data
pods = {}
containers = {}
nodes = {}
error_count = 0
response = api_client.call_api(f"/api/v1/nodes/{node_name}/proxy/metrics/resource", "GET", response_type="str")
data = str(response[0])
rows = data.split("\n")

# Remove comments and empty lines
rows = [row for row in rows if not row.startswith("#") and row.strip() != ""]

# Split lines from rows
rows = [row.split() for row in rows]

for row in rows:
    try:
        if row[0] == "scrape_error":
            error_count = row[1]
        elif row[0][:4] == "node":
            process_node(row, nodes)
        else:
            process_pod_or_container(row, pods, containers)
            
    except IndexError:
        print("Error:",row)

pods_df = pd.DataFrame(pods.values())
pods_df.to_csv("pods.csv", mode='a', header=not os.path.exists("pods.csv"), index=False)

containers_df = pd.DataFrame(containers.values())
containers_df.to_csv("containers.csv", mode='a', header=not os.path.exists("containers.csv"), index=False)

nodes_df = pd.DataFrame(nodes.values())
nodes_df.to_csv("nodes.csv", mode='a', header=not os.path.exists("nodes.csv"), index=False)

error_count_df = pd.DataFrame({"error_count": [error_count], "timestamp": [time.time()]})
error_count_df.to_csv("error_count.csv", mode='a', header=not os.path.exists("error_count.csv"), index=False)

print("Done")

Done


In [13]:
pods_df

,namespace,pod,cpu,cpu_timestamp,memory,memory_timestamp
0,kube-system,coredns-5d78c9869d-lcldh,9.010444,1712038806142,3.2272384e+07,1712038806142
1,kube-system,etcd-minikube,90.535294,1712038811187,8.3206144e+07,1712038811187
2,kube-system,kube-apiserver-minikube,184.354043,1712038801161,3.48209152e+08,1712038801161
3,kube-system,kube-controller-manager-minikube,66.695247,1712038812912,1.04742912e+08,1712038812912
4,kube-system,kube-proxy-p8wcq,2.06375,1712038808434,3.039232e+07,1712038808434
5,kube-system,kube-scheduler-minikube,13.053832,1712038805998,5.6971264e+07,1712038805998
6,kube-system,metrics-server-7746886d4f-cnvs4,14.608032,1712038813346,3.0625792e+07,1712038813346
7,kube-system,storage-provisioner,7.651559,1712038809859,3.5463168e+07,1712038809859
8,social-network,compose-post-service-866f6d7b74-hl962,0.060319,1712038811274,1.458176e+06,1712038811274
9,social-network,home-timeline-redis-7994f7cb85-d89c9,9.080595,1712038796838,7.04512e+06,1712038796838
